In [ ]:
# CELL 0 – Imports and physical constants

import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u   # ← THIS is what was missing
from scipy.interpolate import griddata

# Physical constants
G   = 6.67430e-11         # m^3 kg^-1 s^-2
c   = 2.99792458e8        # m/s
M_sun = 1.98847e30        # kg
kpc = 3.085677581e19      # m

# (optional) cosmology object if you need it later
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
# ================================================================
# CELL 1 — Lens configuration input block for SDSS J1004+4112
# ================================================================
# Change ONLY this cell for each lens.

# 1. Redshifts (lens & source)
z_d = 0.68      # cluster redshift
z_s = 1.734     # quasar redshift

# 2. Angular-diameter distances using CELL 0 cosmology
D_d  = cosmo.angular_diameter_distance(z_d)            # [Mpc]
D_s  = cosmo.angular_diameter_distance(z_s)            # [Mpc]
D_ds = cosmo.angular_diameter_distance_z1z2(z_d, z_s)  # [Mpc]

# --- convert to metres and build time-delay distance factor ---
D_d_m  = D_d.to(u.m).value
D_s_m  = D_s.to(u.m).value
D_ds_m = D_ds.to(u.m).value

# Time-delay distance factor: (1+z_d) D_d D_s / D_ds   [metres]
time_delay_distance_factor = (1.0 + z_d) * (D_d_m * D_s_m / D_ds_m)

print(f"(1+z_d) D_d D_s / D_ds = {time_delay_distance_factor:.3e} m")

# 3. Effective Einstein radius (arcsec)
#    Total separation ~14.6", so θ_E ≈ 7.3" as a rough effective radius
theta_E_arcsec = 7.3

# 4. Main cluster/BCG mass ellipticity and orientation (approximate)
axis_ratio_q       = 0.80      # cluster/BCG mass quite round
position_angle_phi = 30.0      # degrees East of North (rough PA)

# 5. External shear (to represent rest of cluster structure)
#    These are ballpark values; detailed work would refine them.
gamma_ext     = 0.15           # shear amplitude
phi_gamma_deg = 45.0           # shear PA in degrees

# 6. External convergence (LOS mass sheet beyond the modeled cluster)
#    For now we treat the cluster itself as the main lens and set κ_env ~ 0.
kappa_env = 0.00

# 7. Image positions (arcsec, lens-centered) – approximate ring configuration
#    For a first SFH test, we only need them roughly on a ~7–8" ring.
#    Later we can replace with exact astrometry from the literature.
image_positions = {
    "A": ( +7.2,  +0.5 ),
    "B": ( -7.4,  -0.3 ),
    "C": ( +0.6,  +7.6 ),
    "D": ( -0.5,  -7.1 ),
}

# 8. MGE parameters (shape only — for the BCG+cluster core)
#    Broad Gaussians to mimic the large cluster-scale halo.
MGE_sigmas_arcsec = [
    0.8, 1.5, 3.0, 5.0, 8.0,
    12.0, 18.0, 25.0, 35.0
]

MGE_amps = [
    8.0, 6.0, 4.0, 2.5, 1.5,
    0.8, 0.3, 0.15, 0.05
]

(1+z_d) D_d D_s / D_ds = 1.551e+26 m


In [ ]:
# ================================================================
# CELL 2 — Build grid & MGE model (cluster-capable version)
# ================================================================

arcsec_to_rad = (np.pi / 180.0) / 3600.0

# For galaxy lenses, ±5" was enough, but SDSS J1004+4112 is a CLUSTER lens
# with images out at ~7–8". We use a larger grid so all images are covered.
# You can safely keep this for all lenses; it just gives you a bigger field.
x_vals_arcsec = np.linspace(-30.0, 30.0, 800)
y_vals_arcsec = np.linspace(-30.0, 30.0, 800)
x_grid_arcsec, y_grid_arcsec = np.meshgrid(x_vals_arcsec, y_vals_arcsec)

def elliptical_gaussian(x, y, sigma, amp, q, phi_deg):
    """
    2D elliptical Gaussian in angular coordinates (arcsec).
    sigma: width in arcsec
    amp:   amplitude (dimensionless, shape only)
    q:     axis ratio b/a
    phi_deg: position angle (deg, East of North)
    """
    phi = np.deg2rad(phi_deg)
    cosp, sinp = np.cos(phi), np.sin(phi)
    x_rot =  cosp * x + sinp * y
    y_rot = -sinp * x + cosp * y
    r2 = x_rot**2 + (y_rot / q)**2
    return amp * np.exp(-0.5 * r2 / sigma**2)

def psi_norm(x, y):
    """
    Normalized SFH lens potential shape ψ_norm(x, y) from the MGE.
    Absolute normalization is fixed later by the Einstein radius.
    """
    psi = np.zeros_like(x)
    for sigma, amp in zip(MGE_sigmas_arcsec, MGE_amps):
        psi += elliptical_gaussian(
            x, y, sigma, amp,
            axis_ratio_q, position_angle_phi
        )
    return psi

# Base (shape-only) projected potential for the mass distribution
psi_mass_norm = psi_norm(x_grid_arcsec, y_grid_arcsec)

In [ ]:
# CELL 3 — Normalize ψ via |∇ψ| = θ_E (does not change)

# Gradients
dpsi_dy_arcsec, dpsi_dx_arcsec = np.gradient(
    psi_mass_norm,
    y_vals_arcsec,
    x_vals_arcsec
)

# convert to radian deflection
dpsi_dx_rad = dpsi_dx_arcsec / arcsec_to_rad
dpsi_dy_rad = dpsi_dy_arcsec / arcsec_to_rad
alpha_mag = np.sqrt(dpsi_dx_rad**2 + dpsi_dy_rad**2)

# Einstein ring mask
theta_E_rad = theta_E_arcsec * arcsec_to_rad
r_grid = np.sqrt(x_grid_arcsec**2 + y_grid_arcsec**2)
ring_mask = np.abs(r_grid - theta_E_arcsec) <= 0.1

mean_defl_norm = alpha_mag[ring_mask].mean()
A_scale = theta_E_rad / mean_defl_norm

psi_mass_phys = psi_mass_norm * A_scale

In [ ]:
# CELL 4 — Add external shear (does not change)

phi_g = np.deg2rad(phi_gamma_deg)
cos2phi = np.cos(2*phi_g)
sin2phi = np.sin(2*phi_g)

x_rad = x_grid_arcsec * arcsec_to_rad
y_rad = y_grid_arcsec * arcsec_to_rad

psi_shear = 0.5*gamma_ext * (
    (x_rad**2 - y_rad**2)*cos2phi +
    2*x_rad*y_rad*sin2phi
)

psi_total = psi_mass_phys + psi_shear

In [ ]:
# ================================================================
# CELL 5 — Evaluate ψ_SFH at ALL image positions & compute all Δt
# ================================================================

def psi_at(theta_x, theta_y):
    """Interpolate ψ_SFH at an image position."""
    return griddata(
        (x_grid_arcsec.ravel(), y_grid_arcsec.ravel()),
        psi_total.ravel(),
        (theta_x, theta_y),
        method='cubic'
    )

# --- 1. Compute ψ for every image ---
psi_values = {}
for label, (x, y) in image_positions.items():
    psi_values[label] = psi_at(x, y)

print("ψ values at image positions:")
for k, v in psi_values.items():
    print(f"  ψ_{k} = {v:.6e}")
print()

# --- 2. Compute all pairwise delays ---
labels = list(image_positions.keys())
print("Pairwise SFH Time Delays:\n")

for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        Li, Lj = labels[i], labels[j]
        psi_i = psi_values[Li]
        psi_j = psi_values[Lj]

        dpsi = psi_j - psi_i
        dt_raw = (dpsi * time_delay_distance_factor / c) / (24*3600)
        dt_corr = dt_raw / (1 - kappa_env)

        print(f"{Lj} – {Li}:")
        print(f"   Δψ = {dpsi:.6e}")
        print(f"   Raw SFH Δt = {dt_raw:.2f} days")
        print(f"   κ_env-corrected = {dt_corr:.2f} days\n")

ψ values at image positions:
  ψ_A = 1.006633e-09
  ψ_B = 9.604333e-10
  ψ_C = 8.070047e-10
  ψ_D = 8.821568e-10

Pairwise SFH Time Delays:

B – A:
   Δψ = -4.620000e-11
   Raw SFH Δt = -276.56 days
   κ_env-corrected = -276.56 days

C – A:
   Δψ = -1.996287e-10
   Raw SFH Δt = -1195.00 days
   κ_env-corrected = -1195.00 days

D – A:
   Δψ = -1.244766e-10
   Raw SFH Δt = -745.13 days
   κ_env-corrected = -745.13 days

C – B:
   Δψ = -1.534287e-10
   Raw SFH Δt = -918.44 days
   κ_env-corrected = -918.44 days

D – B:
   Δψ = -7.827656e-11
   Raw SFH Δt = -468.57 days
   κ_env-corrected = -468.57 days

D – C:
   Δψ = 7.515210e-11
   Raw SFH Δt = 449.87 days
   κ_env-corrected = 449.87 days

